In [163]:
import pulp as pl

# Definimo las dimensiones del tablero y los barcos
barcos = [3, 1, 1, 2, 4, 2, 1, 2, 1, 3]  # Longitudes de los barcos
requisitos_filas = [3, 2, 2, 4, 2, 1, 1, 2, 3, 0]  # Casilleros ocupados por fila
requisitos_columnas = [1, 2, 1, 3, 2, 2, 3, 1, 5, 0]  # Casilleros ocupados por columna

n = len(requisitos_filas)
m = len(requisitos_columnas)
k = len(barcos)

modelo = pl.LpProblem("batalla_naval", pl.LpMaximize)


# Restricciones


### Restricciones de barcos

**El barco se encuentra posicionado**

Modelo un tablero por cada barco, donde $bx_{i,j}$ representa que el casillero (i, j) está ocupado por
el barco x.

Para todo barco $b_x$ se debe cumplir que ocupa todos los casilleros que debe:

$\sum_{i=0}^{n-1} \sum_{j=0}^{m-1} bx_{i,j} = B_x - 1\quad \forall x$, $B_x$ = largo del barco x.

El -1 es porque el casillero inicial del barco se modela con la variable $sx_{i,j}$.

In [ ]:
bx = [[[pl.LpVariable(f"b_{x}_{i}_{j}", cat='Binary') for j in range(m)] for i in range(n)] for x in range(k)]

for x in range(k):
    # El -1 es porque sx representa el primer casillero
    modelo += pl.lpSum(bx[x][i][j] for i in range(n) for j in range(m)) == barcos[x] - 1


**Adyacencia horizontal o vertical**

Lo primero es definir una variable que me indique si el barco x se encuentra posicionado horizontal o
verticalmente.

Para eso defino $h_x$ como una variable booleana que si vale:
* **1**: el barco x está posicionado horizontalmente
* **0**: el barco x está posicionado verticalmente

In [158]:
hx = [pl.LpVariable(f"h_{x}", cat='Binary') for x in range(k)]



Defino ${sx}_{i,j}$ como el casillero donde empieza el barco x. ${sx}_{i,j}$
es una variable booleana y cumple que:

$\sum_{i=0}^{n-1} \sum_{j=0}^{m-1} sx_{i,j} = 1 \quad \forall x$

Entonces, se tiene que cumplir que:

**Caso horizontal**: 

$ {sx}_{i,j} + \sum_{col=j+1}^{j+B_x-1} {bx}_{i,col} \geq B_x \cdot h_x - M \cdot {sx}_{i,j}$

**Caso vertical**: 

$ {sx}_{i,j} + \sum_{fil=i+1}^{i+B_x-1} {bx}_{fil,j} \geq B_x \cdot (1 - h_x) - M \cdot {sx}_{i,j}$

In [159]:
sx = [[[pl.LpVariable(f"s_{x}_{i}_{j}", cat='Binary') for j in range(m)] for i in range(n)] for x in range(k)]


for x in range(k):
    modelo += pl.lpSum(sx[x][i][j] for i in range(n) for j in range(m)) == 1

M = 100000

for x in range(k):
    Bx = barcos[x]
    for i in range(n - Bx + 1):  # Evitar overflow
        for j in range(m - Bx + 1):  # Evitar overflow
            # Caso horizontal
            modelo += sx[x][i][j] + pl.lpSum(bx[x][i][col] for col in range(j + 1, j + Bx)) >= Bx * hx[x] - M * sx[x][i][j]

            # Caso vertical
            modelo += sx[x][i][j] + pl.lpSum(bx[x][fil][j] for fil in range(i + 1, i + Bx)) >= Bx * (1 - hx[x]) - M * sx[x][i][j]


### Restricciones de casillero

$c_{i,j}$: el casillero (i,j) se encuentra ocupado

$c_{i,j} = \sum_{x=1}^{k} {sx}_{i,j} + \sum_{x=1}^{k} {bx}_{i,j} \leq 1$

Esto fuerza a que el casillero sea 0 o 1 y además fuerza a que no se puedan superponer los barcos.

In [160]:
casilleros = [
    [pl.LpVariable(f"c{i},{j}", cat="Binary") for j in range(m)] for i in range(n)
]

for i in range(n):
    for j in range(m):
        modelo += casilleros[i][j] == pl.lpSum(
            sx[x][i][j] for x in range(k)
        ) + pl.lpSum(
            bx[x][i][j] for x in range(k)
        )
        
        modelo += casilleros[i][j] <= 1 

### Requisito consumo de filas y columnas
$\sum_{j=0}^{m-1} c_{i,j} = F_i \quad \forall i \in \{0, 1, \dots, n-1\}$

$\sum_{i=0}^{n-1} c_{i,j} = C_j \quad \forall j \in \{0, 1, \dots, m-1\}$

Donde $F_i$ y $C_j$ representan el consumo de la fila i y la columna j respectivamente.



In [161]:
# Respetar los requisitos por fila
for i in range(n):
    modelo += pl.lpSum(casilleros[i][j] for j in range(m)) == requisitos_filas[i]

# Respetar los requisitos por columna
for j in range(m):
    modelo += pl.lpSum(casilleros[i][j] for i in range(n)) == requisitos_columnas[j]

### Requisito adyacencia

...

### Resuelvo

In [162]:
# Ecuación a maximizar (ocupar todos los casilleros que pueda) -------------------------------------
modelo += pl.lpSum(casilleros[i][j] for i in range(n) for j in range(m))

modelo.solve(pl.PULP_CBC_CMD(msg=False)) # Muteo el print de pulp

# Mostrar el mapa del tablero ----------------------------------------------------------------------
# if pl.LpStatus[modelo.status] == "Optimal":
tablero = [[" " for _ in range(m)] for _ in range(n)]
for i in range(n):
    for j in range(m):
        if pl.value(casilleros[i][j]) == 1:
            tablero[i][j] = "■"

print("Tablero:")
print("  " + " ".join(map(str, requisitos_columnas)))
for i, fila in enumerate(tablero):
    print(
        f"{requisitos_filas[i]} " + " ".join(fila)
    )
# # else:
# #     print("No se encontró una solución óptima.")

Tablero:
  1 2 1 3 2 2 3 1 5 0
3 ■ ■         ■      
2   ■             ■  
2           ■     ■  
4     ■ ■ ■ ■        
2       ■         ■  
1                 ■  
1                 ■  
2             ■      
3       ■ ■   ■   ■  
0                    
